In [1]:
import warnings
warnings.filterwarnings('ignore')

In [13]:
import pandas as pd
import numpy as np
import time
from datetime import datetime, timedelta
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

In [3]:
df = pd.read_parquet(f"data/ADAUSDT.parquet", engine='pyarrow')

In [4]:
df

,open,high,low,close,volume,price_change,next_log_return,high_low_spread,close_open_spread,volume_change,...,rolling_max_10,rolling_min_10,rolling_mean_20,rolling_max_20,rolling_min_20,Skew_20,kurtosis_20,zscore_20,day_of_week,hour_of_day
datetime,,,,,,,,,,,,,,,,,,,,,
2021-08-01 00:00:00,1.3193,1.3196,1.3114,1.3149,304283.0,0.0000,1.314481,0.0082,-0.0044,0.0,...,1.3456,1.3149,1.336325,1.3495,1.3149,-0.713390,-0.879934,0.330574,6,0
2021-08-01 00:05:00,1.3149,1.3192,1.3149,1.3174,80684.0,0.0025,0.001899,0.0043,0.0025,-223599.0,...,1.3456,1.3149,1.336325,1.3495,1.3149,-0.713390,-0.879934,0.330574,6,0
2021-08-01 00:10:00,1.3174,1.3203,1.3138,1.3156,86340.0,-0.0018,-0.001367,0.0065,-0.0018,5656.0,...,1.3456,1.3149,1.336325,1.3495,1.3149,-0.713390,-0.879934,0.330574,6,0
2021-08-01 00:15:00,1.3156,1.3222,1.3149,1.3222,35944.0,0.0066,0.005004,0.0073,0.0066,-50396.0,...,1.3456,1.3149,1.336325,1.3495,1.3149,-0.713390,-0.879934,0.330574,6,0
2021-08-01 00:20:00,1.3222,1.3273,1.3222,1.3270,48829.0,0.0048,0.003624,0.0051,0.0048,12885.0,...,1.3456,1.3149,1.336325,1.3495,1.3149,-0.713390,-0.879934,0.330574,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-10-13 13:10:00,0.3529,0.3531,0.3526,0.3527,127166.0,-0.0002,-0.000567,0.0005,-0.0002,59477.0,...,0.3537,0.3523,0.352795,0.3537,0.3520,0.413726,0.623319,-0.237098,6,13
2024-10-13 13:15:00,0.3527,0.3532,0.3526,0.3532,74899.0,0.0005,0.001417,0.0006,0.0005,-52267.0,...,0.3532,0.3523,0.352800,0.3537,0.3520,0.391377,0.472470,0.987747,6,13
2024-10-13 13:20:00,0.3532,0.3535,0.3530,0.3531,654819.0,-0.0001,-0.000283,0.0005,-0.0001,579920.0,...,0.3532,0.3523,0.352835,0.3537,0.3520,0.200778,0.524056,0.664012,6,13


In [5]:
df['target'] = df['close'].shift(-1)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 336827 entries, 2021-08-01 00:00:00 to 2024-10-13 13:30:00
Data columns (total 49 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   open               336827 non-null  float32
 1   high               336827 non-null  float32
 2   low                336827 non-null  float32
 3   close              336827 non-null  float32
 4   volume             336827 non-null  float32
 5   price_change       336827 non-null  float32
 6   next_log_return    336827 non-null  float32
 7   high_low_spread    336827 non-null  float32
 8   close_open_spread  336827 non-null  float32
 9   volume_change      336827 non-null  float32
 10  OBV                336827 non-null  float32
 11  rolling_std        336827 non-null  float32
 12  upper_band         336827 non-null  float32
 13  lower_band         336827 non-null  float32
 14  ATR                336827 non-null  float32
 15  RSI              

In [7]:
df = df.drop(columns=['support_line', 'resistance_line', 'tenkan_sen', 'kijun_sen', 'senkou_span_a', 'senkou_span_b'])

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 336827 entries, 2021-08-01 00:00:00 to 2024-10-13 13:30:00
Data columns (total 43 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   open               336827 non-null  float32
 1   high               336827 non-null  float32
 2   low                336827 non-null  float32
 3   close              336827 non-null  float32
 4   volume             336827 non-null  float32
 5   price_change       336827 non-null  float32
 6   next_log_return    336827 non-null  float32
 7   high_low_spread    336827 non-null  float32
 8   close_open_spread  336827 non-null  float32
 9   volume_change      336827 non-null  float32
 10  OBV                336827 non-null  float32
 11  rolling_std        336827 non-null  float32
 12  upper_band         336827 non-null  float32
 13  lower_band         336827 non-null  float32
 14  ATR                336827 non-null  float32
 15  RSI              

In [30]:
df.isna().sum()

open                 0
high                 0
low                  0
close                0
volume               0
price_change         0
next_log_return      0
high_low_spread      0
close_open_spread    0
volume_change        0
OBV                  0
rolling_std          0
upper_band           0
lower_band           0
ATR                  0
RSI                  0
MACD                 0
Signal_Line          0
SMA                  0
EMA                  0
ROC                  0
+DI                  0
-DI                  0
ADX                  0
chikou_span          0
lag_1                0
lag_2                0
lag_3                0
rolling_mean_5       0
rolling_max_5        0
rolling_min_5        0
rolling_mean_10      0
rolling_max_10       0
rolling_min_10       0
rolling_mean_20      0
rolling_max_20       0
rolling_min_20       0
Skew_20              0
kurtosis_20          0
zscore_20            0
day_of_week          0
hour_of_day          0
target               1
dtype: int6

In [8]:
df = df.dropna()

In [9]:
X = df.drop(columns=['target'])  # Замените 'target_column' на название целевой переменной
y = df['target']

In [34]:
tscv = TimeSeriesSplit(n_splits=10)

In [39]:
for train_index, test_index in tscv.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    
    # Предсказание и оценка модели
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    print(f'MAE для текущего разбиения: {mae}')

MAE для текущего разбиения: 0.09913312715568393


KeyboardInterrupt: 

In [10]:
# Определяем пропорцию 70/30
train_size = 0.7  # 70% для тренировки

# Определяем индекс для разбиения
split_index = int(len(df) * train_size)

# Тренировочные данные: первые 70% данных
X_train, y_train = X.iloc[:split_index], y.iloc[:split_index]

# Тестовые данные: оставшиеся 30% данных
X_test, y_test = X.iloc[split_index:], y.iloc[split_index:]

In [14]:
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Предсказание и оценка модели
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'MAE для текущего разбиения: {mae}')

KeyboardInterrupt: 

In [ ]:
plt.plot(y_test, label='Actual Prices')
plt.plot(y_pred, label='Predicted Prices')
plt.legend()
plt.show()


In [ ]:
fig, axes = plt.subplots(nrows=2, figsize=(14, 8), sharex=True)

df_transformed['2010':].ip.plot(ax=axes[0], label='actual', title='Liquor')
preds.ip.plot(label='predicted', ax=axes[0])
axes[0].legend()

df_transformed['2010':].sentiment.plot(ax=axes[1], label='actual', title='Sentiment')
preds.sentiment.plot(label='predicted', ax=axes[1])
axes[1].legend()
axes[1].set_xlabel('')
sns.despine()
fig.tight_layout();